# Make a Coadd from Gen3 Run3 DDF



See Bob Armstrong slides

https://docs.google.com/presentation/d/1bs3z6bqpAZ2WuSjpZOGuhgRnpz3Zd7ighkM-IDgk1dA/view

Run 2.2i DC2 data in:
/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo
Create a user directory in the gen3 repo. This should be a symlink to write-accessible area [that you’ve created separately]:
/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo/u/<username>
Ask Jim for credentials to access to the postgres db

Run 3.1i DC2 DDF data in:

/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/

1. Create your own user directory under the above repo
    
/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/u/

E.g., /global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/u/wmwv
    
as a symlink to a directory you've created on $CSCRATCH to hold data.

```
mkdir -p $SCRATCH/DC2/Gen3/Run_3.1i/repo
ln -s $SCRATCH/DC2/Gen3/Run_3.1i/repo /global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/u/wmwv
```



Bob's slide on Learning the new Butler

Science Pipeline Tutorial from DM: https://pipelines.lsst.io/v/weekly/

Middleware tutorials: https://pipelines.lsst.io/v/weekly/middleware/faq.html, https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/organizing.html

Testing datasets: https://github.com/lsst/ci_imsim - A small dataset that allows you to run through the whole pipeline in a finite set of time.

Slack channels: #dm-middleware-support, #dm-middleware-dev

Tutorial by Jim Chiang on how to use parsl to scale processing on NERSC: https://docs.google.com/presentation/d/1EO_UBVhISBrBussCsIvJhNVxSnyfg5z6yIPaz99gA0A/view

Tutorial on creating a PipelineTask: https://github.com/lsst/pipe_base/blob/main/doc/lsst.pipe.base/creating-a-pipelinetask.rst


In [ ]:
from lsst.daf.butler import Butler

In [ ]:
butler = Butler("/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/")

In [ ]:
list(butler.registry.queryDatasetTypes())

A careful parsing of the above list will reveal that there are no coadds available yet (as of 2022-04-12).  Just single-visit processing.

In [ ]:
list(butler.registry.queryDatasetTypes("goodSeeingCoadd"))

One key dataset we're going to need from the single-frame processing is the `visitSummary`

In [ ]:
list(butler.registry.queryDatasetTypes("visitSummary"))

From the `visitSummary` we can get our visit list for a given tract, patch

Make a YAML file describing what we want to happen
subsets:
    dia
    - selectListVisits
    - templateGen


In [ ]:
list(butler.registry.queryCollections())

In [ ]:
i = 0
for data_id in butler.registry.queryDataIds(
    ["tract", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="VisitSummary",
    collections="u/descdm/sfp_ddf_visits_part_01",
    visit="249460",
):
    i = i+1
    if i > 10:
        break
        
    print(data_id)

I don't understand why there are duplicate entries.

In [ ]:
i = 0
for data_id in butler.registry.queryDataIds(
    ["tract", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="VisitSummary",
    collections="u/descdm/sfp_ddf_visits_part_01",
    tract="4848",
):
    print(i)
    i = i+1
    if i > 10:
        break
        
    print(data_id)

In [ ]:
from collections import defaultdict
grouped_by_tract = defaultdict(set)

i = 0
for data_id in butler.registry.queryDataIds(
    ["tract", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="VisitSummary",
    collections="u/descdm/sfp_ddf_visits_part_01",
):  
    i = i+1
    if i > 1000:
        break

    grouped_by_tract[data_id["tract"]].add(data_id)
    
print({k: len(v) for k, v in grouped_by_tract.items()})
                                            

The LSST Science Pipelines Data Release Processing:

https://github.com/lsst/drp_pipe
https://github.com/lsst/drp_pipe/blob/main/ingredients/DRP.yaml


From Lee Kelvin's writeup:  

"""  
Coadd-processing takes place within step3 of the DRP.yaml pipeline. A large number of tasks are performed during this step, including, but not limited to:

jointcal - astrometric and photometric calibration  
makeWarp - warping of visit-level imaging onto the coadd-plane  
assembleCoadd - coaddition of warped visit-level imaging into coadded images  
detection - coadd-level object detection  
deblend - object deblending  
writeObjectTable - construction of coadd-level object catalogue  
"""

You can use the command-line version of the `butler` query tools by launcing a matching environment:

On the command line:
```
shifter --clearenv --image=lsstdesc/td-env:dev /bin/bash
bash-4.2$ source /opt/lsst/software/stack/loadLSST.bash
ebash-4.2$ setup lsst_distrib
```

```
REPO=/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo
```